# v1.3 Zero Label Strategy Analysis

v1.3プロンプトで選定された銘柄の分析

## v1.3の特徴
- カテゴリーフィールドを完全削除（Zero Label戦略）
- Twitter言及数: 80件以上は除外、20-50件を最適ゾーンとして優先
- 前日騰落率: +16%以上は警戒、+13-14%は安全
- 複合判定: Twitter × 前日騰落率を組み合わせた判定

In [48]:
import sys
from pathlib import Path
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Project root
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

## 1. Data Loading

In [49]:
# Load v1.3 archive WITH backtest data
archive_path = project_root / "data" / "parquet" / "backtest" / "v1.3_grok_trending_archive_with_backtest.parquet"
df = pd.read_parquet(archive_path)

print(f"Total stocks: {len(df)}")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
print(f"Stocks with backtest data: {df['phase2_return'].notna().sum()}")
print(f"\nBacktest columns:")
backtest_cols = [col for col in df.columns if 'buy' in col or 'sell' in col or 'phase' in col or 'backtest' in col or 'profit' in col or 'selection' in col]
for col in backtest_cols:
    print(f"  - {col}")

Total stocks: 25
Date range: 2025-11-05 to 2025-11-06
Stocks with backtest data: 25

Backtest columns:
  - selection_score
  - selection_rank
  - selection_date
  - backtest_date
  - buy_price
  - sell_price
  - phase1_return
  - phase1_win
  - profit_per_100_shares_phase1
  - phase2_return
  - phase2_win
  - profit_per_100_shares_phase2


In [50]:
# Basic statistics
df.describe()

,sentiment_score,selection_score,previous_day_change_pct,twitter_mentions,grok_rank,selection_rank,buy_price,sell_price,daily_close,high,low,volume,phase1_return,profit_per_100_shares_phase1,phase2_return,profit_per_100_shares_phase2
count,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.00000,25.000000,2.500000e+01,25.000000,25.000000,25.000000,25.000000
mean,0.660000,78.400000,3.488400,16.520000,7.000000,7.000000,1852.360000,1851.020000,1851.020000,1883.66000,1804.620000,1.396552e+06,0.008549,-134.000000,0.008549,-134.000000
std,0.043301,5.866572,7.074082,15.478803,4.082483,4.082483,2066.519797,2052.105415,2052.105415,2096.04979,2002.801503,3.050206e+06,0.054511,6555.988357,0.054511,6555.988357
min,0.550000,70.000000,-1.570000,0.000000,1.000000,1.000000,110.000000,109.000000,109.000000,110.00000,107.000000,2.100000e+03,-0.054241,-16050.000000,-0.054241,-16050.000000
25%,0.630000,74.000000,0.000000,0.000000,4.000000,4.000000,480.000000,470.000000,470.000000,480.00000,466.000000,2.080000e+04,-0.011735,-1000.000000,-0.011735,-1000.000000
50%,0.670000,78.000000,1.250000,20.000000,7.000000,7.000000,914.000000,914.000000,914.000000,914.00000,914.000000,9.760000e+04,-0.001220,-100.000000,-0.001220,-100.000000
75%,0.700000,82.000000,2.100000,28.000000,10.000000,10.000000,2625.000000,2595.000000,2595.000000,2645.00000,2540.000000,7.864000e+05,0.007075,600.000000,0.007075,600.000000
max,0.720000,90.000000,25.910000,45.000000,15.000000,15.000000,8200.000000,8190.000000,8190.000000,8370.00000,7930.000000,1.157580e+07,0.242152,23100.000000,0.242152,23100.000000


## 2. Twitter Mentions Distribution

In [51]:
# Twitter mentions histogram
fig = px.histogram(
    df,
    x="twitter_mentions",
    nbins=20,
    title="Twitter Mentions Distribution (v1.3)",
    labels={"twitter_mentions": "Twitter Mentions"},
    color_discrete_sequence=["steelblue"]
)

# Add vertical lines for optimal zones
fig.add_vline(x=20, line_dash="dash", line_color="green", annotation_text="Optimal Start (20)")
fig.add_vline(x=50, line_dash="dash", line_color="green", annotation_text="Optimal End (50)")
fig.add_vline(x=80, line_dash="dash", line_color="red", annotation_text="Danger Zone (80+)")

fig.update_layout(height=500)
fig.show()

## 3. Previous Day Change Distribution

In [52]:
# Previous day change histogram
fig = px.histogram(
    df,
    x="previous_day_change_pct",
    nbins=30,
    title="Previous Day Change Distribution (v1.3)",
    labels={"previous_day_change_pct": "Previous Day Change (%)"},
    color_discrete_sequence=["coral"]
)

# Add vertical lines for thresholds
fig.add_vline(x=13, line_dash="dash", line_color="green", annotation_text="Safe Start (13%)")
fig.add_vline(x=14, line_dash="dash", line_color="green", annotation_text="Safe End (14%)")
fig.add_vline(x=16, line_dash="dash", line_color="red", annotation_text="Danger (16%+)")

fig.update_layout(height=500)
fig.show()

## 4. Twitter vs Previous Day Change (Scatter)

In [53]:
# Scatter plot with color by date
fig = px.scatter(
    df,
    x="twitter_mentions",
    y="previous_day_change_pct",
    color="date",
    hover_data=["stock_name", "ticker"],
    title="Twitter Mentions vs Previous Day Change (v1.3)",
    labels={
        "twitter_mentions": "Twitter Mentions",
        "previous_day_change_pct": "Previous Day Change (%)"
    },
    size_max=10
)

# Add zones
# Optimal zone (20-50 mentions)
fig.add_vrect(x0=20, x1=50, fillcolor="green", opacity=0.1, annotation_text="Optimal Zone")

# Danger zone (80+ mentions or 16%+ change)
fig.add_vrect(x0=80, x1=df['twitter_mentions'].max() + 10, fillcolor="red", opacity=0.1)
fig.add_hrect(y0=16, y1=df['previous_day_change_pct'].max() + 5, fillcolor="red", opacity=0.1, annotation_text="Danger Zone")

fig.update_layout(height=600)
fig.show()

## 5. Stocks by Date

In [54]:
# Count by date
date_counts = df.groupby('date').size().reset_index(name='count')

fig = px.bar(
    date_counts,
    x='date',
    y='count',
    title="Number of Stocks Selected by Date (v1.3)",
    labels={"date": "Selection Date", "count": "Number of Stocks"},
    text='count',
    color_discrete_sequence=["teal"]
)

fig.update_layout(height=400)
fig.show()

## 6. Detailed Table View

In [55]:
# Show key columns
display_cols = [
    'date', 'ticker', 'stock_name', 
    'twitter_mentions', 'previous_day_change_pct',
    'Close', 'Volume', 'rsi14'
]

df_display = df[display_cols].sort_values(['date', 'twitter_mentions'])
df_display

,date,ticker,stock_name,twitter_mentions,previous_day_change_pct,Close,Volume,rsi14
7,2025-11-05,4746.T,東計電算,12,4.30,None,None,None
1,2025-11-05,8059.T,第一実業,15,0.57,None,None,None
5,2025-11-05,4388.T,エーアイ,20,-0.62,None,None,None
2,2025-11-05,7089.T,フォースタートアップス,21,1.00,None,None,None
8,2025-11-05,3123.T,サイボー,22,1.20,None,None,None
4,2025-11-05,3110.T,日東紡績,24,2.50,None,None,None
11,2025-11-05,7760.T,IMV,25,-0.65,None,None,None
3,2025-11-05,3744.T,サイオス,26,1.50,None,None,None
0,2025-11-05,6927.T,ヘリオステクノホールディングス,28,2.10,None,None,None
9,2025-11-05,4477.T,BASE,30,0.00,None,None,None


## 7. Statistics by Date

In [56]:
# Group by date statistics
stats_by_date = df.groupby('date').agg({
    'twitter_mentions': ['count', 'min', 'max', 'mean'],
    'previous_day_change_pct': ['min', 'max', 'mean']
}).round(2)

print("Statistics by Date:")
print(stats_by_date)

Statistics by Date:
           twitter_mentions                previous_day_change_pct         \
                      count min max   mean                     min    max   
date                                                                        
2025-11-05               15  12  45  27.53                   -0.65   4.30   
2025-11-06               10   0   0   0.00                   -1.57  25.91   

                  
            mean  
date              
2025-11-05  1.30  
2025-11-06  6.76  


## 8. Zone Analysis

v1.3戦略で定義されたゾーンに分類

In [57]:
# Classify stocks into zones
def classify_zone(row):
    twitter = row['twitter_mentions']
    prev_chg = row['previous_day_change_pct']
    
    # Danger zones
    if twitter >= 80 or prev_chg >= 16:
        return "Danger"
    # Optimal zone
    elif 20 <= twitter <= 50 and prev_chg < 16:
        return "Optimal"
    # Safe but not optimal
    elif twitter < 80 and prev_chg < 16:
        return "Safe"
    else:
        return "Other"

df['zone'] = df.apply(classify_zone, axis=1)

# Count by zone
zone_counts = df.groupby('zone').size().reset_index(name='count')
zone_counts = zone_counts.sort_values('count', ascending=False)

fig = px.pie(
    zone_counts,
    values='count',
    names='zone',
    title="Stock Distribution by Zone (v1.3)",
    color='zone',
    color_discrete_map={
        'Optimal': 'green',
        'Safe': 'lightblue',
        'Danger': 'red',
        'Other': 'gray'
    }
)

fig.update_layout(height=500)
fig.show()

print("\nZone Distribution:")
print(zone_counts)


Zone Distribution:
      zone  count
1  Optimal     13
2     Safe      9
0   Danger      3


## 9. Export for Further Analysis

In [58]:
# Save enriched data with zone classification
output_path = project_root / "data" / "parquet" / "backtest" / "v1.3_grok_trending_archive_with_zones.parquet"
df.to_parquet(output_path, index=False)
print(f"Saved enriched data to: {output_path}")

Saved enriched data to: /Users/hiroyukiyamanaka/Desktop/python_stock/dash_plotly/data/parquet/backtest/v1.3_grok_trending_archive_with_zones.parquet


## 10. バックテスト結果：v1.3で金が増えるか？

v1.3で選定された銘柄の実際のパフォーマンス（円ベース）

### Phase1とPhase2の定義
- **Phase1**: 寄り付き買い → 前場引け売り（近似: 終値を使用）
- **Phase2**: 寄り付き買い → 大引け売り

### 利益計算
- **100株あたりの利益**を円（¥）で表示
- 例: buy_price=1000円, sell_price=1050円 → 利益=¥5,000/100株

In [59]:
# データ取得状況確認
print(f"Total stocks: {len(df)}")
print(f"Stocks with Phase1 data: {df['phase1_return'].notna().sum()}")
print(f"Stocks with Phase2 data: {df['phase2_return'].notna().sum()}")
print()

# Phase2の基本統計
if df['phase2_return'].notna().sum() > 0:
    phase2_avg_return = df['phase2_return'].mean() * 100
    phase2_avg_profit = df['profit_per_100_shares_phase2'].mean()
    phase2_win_rate = df['phase2_win'].sum() / df['phase2_return'].notna().sum() * 100
    
    print("Phase2（寄り付き → 大引け）サマリー:")
    print(f"  平均リターン: {phase2_avg_return:+.2f}%")
    print(f"  平均利益: ¥{phase2_avg_profit:+,.0f}/100株")
    print(f"  勝率: {phase2_win_rate:.1f}%")
    print(f"  勝ち: {df['phase2_win'].sum()}銘柄")
    print(f"  負け: {(~df['phase2_win']).sum()}銘柄")

Total stocks: 25
Stocks with Phase1 data: 25
Stocks with Phase2 data: 25

Phase2（寄り付き → 大引け）サマリー:
  平均リターン: +0.85%
  平均利益: ¥-134/100株
  勝率: 36.0%
  勝ち: 9銘柄
  負け: 16銘柄


### 100株あたりの利益分布（Phase2）

In [60]:
# Phase2の利益分布（円ベース）
df_phase2 = df[df['phase2_return'].notna()].copy()

fig = px.histogram(
    df_phase2,
    x="profit_per_100_shares_phase2",
    nbins=20,
    title=f"Phase2: 100株あたり利益分布 (平均: ¥{df_phase2['profit_per_100_shares_phase2'].mean():+,.0f})",
    labels={"profit_per_100_shares_phase2": "利益（¥/100株）"},
    color_discrete_sequence=["steelblue"]
)

# 損益分岐点
fig.add_vline(x=0, line_dash="dash", line_color="black", annotation_text="損益分岐点")
fig.add_vline(
    x=df_phase2['profit_per_100_shares_phase2'].mean(), 
    line_dash="solid", 
    line_color="red", 
    annotation_text=f"平均"
)

fig.update_layout(height=500)
fig.show()

In [61]:
# 銘柄別の利益（Phase2、円ベース）
df_sorted = df_phase2.sort_values('profit_per_100_shares_phase2', ascending=True)

# 色分け（利益/損失）
df_sorted['color'] = df_sorted['profit_per_100_shares_phase2'].apply(
    lambda x: 'green' if x > 0 else 'red'
)

fig = px.bar(
    df_sorted,
    x='profit_per_100_shares_phase2',
    y='stock_name',
    orientation='h',
    title="銘柄別利益（Phase2: 寄り付き → 大引け、100株あたり）",
    labels={"profit_per_100_shares_phase2": "利益（¥/100株）", "stock_name": "銘柄"},
    color='color',
    color_discrete_map={'green': 'green', 'red': 'red'},
    hover_data=['ticker', 'backtest_date', 'buy_price', 'daily_close', 'phase2_return']
)

fig.add_vline(x=0, line_dash="dash", line_color="black")
fig.update_layout(height=600, showlegend=False)
fig.update_traces(texttemplate='¥%{x:+,.0f}', textposition='outside')
fig.show()

### リターン率 vs 利益額

In [62]:
# リターン率（%）vs 利益額（¥）の関係
fig = px.scatter(
    df_phase2,
    x='phase2_return',
    y='profit_per_100_shares_phase2',
    size='buy_price',
    color='phase2_win',
    color_discrete_map={True: 'green', False: 'red'},
    hover_data=['stock_name', 'ticker', 'buy_price', 'daily_close'],
    title="Phase2: リターン率 vs 利益額（バブルサイズ=買値）",
    labels={
        "phase2_return": "リターン率",
        "profit_per_100_shares_phase2": "利益（¥/100株）",
        "phase2_win": "勝敗"
    }
)

# 軸フォーマット
fig.update_xaxes(tickformat=".1%")
fig.update_yaxes(tickprefix="¥", tickformat=",")

# 損益分岐点
fig.add_hline(y=0, line_dash="dash", line_color="black")
fig.add_vline(x=0, line_dash="dash", line_color="black")

fig.update_layout(height=600)
fig.show()

### 選定基準と利益の関係

In [63]:
# Twitter言及数 vs 利益額
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Twitter言及数 vs 利益額", "前日騰落率 vs 利益額")
)

# Left: Twitter Mentions
fig.add_trace(
    go.Scatter(
        x=df_phase2['twitter_mentions'],
        y=df_phase2['profit_per_100_shares_phase2'],
        mode='markers',
        marker=dict(
            size=10,
            color=df_phase2['profit_per_100_shares_phase2'],
            colorscale='RdYlGn',
            showscale=True,
            colorbar=dict(title="利益（¥）", x=0.45),
            cmid=0
        ),
        text=df_phase2['stock_name'],
        hovertemplate='<b>%{text}</b><br>Twitter: %{x}<br>利益: ¥%{y:+,.0f}<extra></extra>'
    ),
    row=1, col=1
)

# Right: Previous Day Change
fig.add_trace(
    go.Scatter(
        x=df_phase2['previous_day_change_pct'],
        y=df_phase2['profit_per_100_shares_phase2'],
        mode='markers',
        marker=dict(
            size=10,
            color=df_phase2['profit_per_100_shares_phase2'],
            colorscale='RdYlGn',
            showscale=False,
            cmid=0
        ),
        text=df_phase2['stock_name'],
        hovertemplate='<b>%{text}</b><br>前日: %{x:.2f}%<br>利益: ¥%{y:+,.0f}<extra></extra>'
    ),
    row=1, col=2
)

# 損益分岐点
fig.add_hline(y=0, line_dash="dash", line_color="black", row=1, col=1)
fig.add_hline(y=0, line_dash="dash", line_color="black", row=1, col=2)

# Update axes
fig.update_xaxes(title_text="Twitter言及数", row=1, col=1)
fig.update_xaxes(title_text="前日騰落率（%）", row=1, col=2)
fig.update_yaxes(title_text="利益（¥/100株）", row=1, col=1)
fig.update_yaxes(title_text="利益（¥/100株）", row=1, col=2)

fig.update_layout(height=500, showlegend=False, title_text="Phase2: 選定基準と利益の関係")
fig.show()

### 詳細テーブル

In [64]:
# 詳細バックテスト結果テーブル
backtest_display_cols = [
    'date', 'selection_date', 'backtest_date', 'ticker', 'stock_name',
    'twitter_mentions', 'previous_day_change_pct',
    'buy_price', 'daily_close',
    'profit_per_100_shares_phase2', 'phase2_return', 'phase2_win'
]

df_backtest_display = df_phase2[backtest_display_cols].sort_values('profit_per_100_shares_phase2', ascending=False)

# Style the dataframe
df_backtest_styled = df_backtest_display.style.format({
    'previous_day_change_pct': '{:+.2f}%',
    'buy_price': '¥{:.1f}',
    'daily_close': '¥{:.1f}',
    'profit_per_100_shares_phase2': '¥{:+,.0f}',
    'phase2_return': '{:+.2%}'
}).background_gradient(
    subset=['profit_per_100_shares_phase2'],
    cmap='RdYlGn',
    vmin=-10000,
    vmax=10000
)

df_backtest_styled

,date,selection_date,backtest_date,ticker,stock_name,twitter_mentions,previous_day_change_pct,buy_price,daily_close,profit_per_100_shares_phase2,phase2_return,phase2_win
15,2025-11-06,2025-11-05,2025-11-06,8059.T,第一実業,0,+0.00%,¥2650.0,¥2881.0,"¥+23,100",+8.72%,True
11,2025-11-05,2025-11-04,2025-11-05,7760.T,IMV,25,-0.65%,¥1982.0,¥2050.0,"¥+6,800",+3.43%,True
17,2025-11-06,2025-11-05,2025-11-06,3928.T,マイネット,0,+25.91%,¥223.0,¥277.0,"¥+5,400",+24.22%,True
23,2025-11-06,2025-11-05,2025-11-06,5595.T,QPS研究所,0,-1.57%,¥1970.0,¥1999.0,"¥+2,900",+1.47%,True
0,2025-11-05,2025-11-04,2025-11-05,6927.T,ヘリオステクノホールディングス,28,+2.10%,¥866.0,¥874.0,¥+800,+0.92%,True
22,2025-11-06,2025-11-05,2025-11-06,6158.T,和井田製作所,0,+1.43%,¥848.0,¥854.0,¥+600,+0.71%,True
19,2025-11-06,2025-11-05,2025-11-06,7760.T,IMV,0,+1.37%,¥2072.0,¥2078.0,¥+600,+0.29%,True
3,2025-11-05,2025-11-04,2025-11-05,3744.T,サイオス,26,+1.50%,¥485.0,¥490.0,¥+500,+1.03%,True
8,2025-11-05,2025-11-04,2025-11-05,3123.T,サイボー,22,+1.20%,¥576.0,¥577.0,¥+100,+0.17%,True
16,2025-11-06,2025-11-05,2025-11-06,7069.T,サイバー・バズ,0,+19.63%,¥914.0,¥914.0,¥+0,+0.00%,False


### ゾーン別パフォーマンス分析

In [65]:
# 最終結論
print("=" * 60)
print("v1.3 Zero Label Strategy: バックテスト結果サマリー")
print("=" * 60)
print()
print(f"検証期間: 2025-11-05, 2025-11-06（2日分）")
print(f"検証銘柄数: {len(df_phase2)}銘柄")
print()
print("Phase2（寄り付き買い → 大引け売り）:")
print(f"  平均リターン: {df_phase2['phase2_return'].mean()*100:+.2f}%")
print(f"  平均利益: ¥{df_phase2['profit_per_100_shares_phase2'].mean():+,.0f}/100株")
print(f"  勝率: {(df_phase2['phase2_win'].sum() / len(df_phase2) * 100):.1f}% ({df_phase2['phase2_win'].sum()}勝{(~df_phase2['phase2_win']).sum()}敗)")
print()
print(f"全{len(df_phase2)}銘柄に100株ずつ投資した場合:")
print(f"  合計損益: ¥{df_phase2['profit_per_100_shares_phase2'].sum():+,.0f}")
print()
print("ベスト3銘柄:")
top3 = df_phase2.nlargest(3, 'profit_per_100_shares_phase2')[['stock_name', 'profit_per_100_shares_phase2', 'phase2_return']]
for idx, (_, row) in enumerate(top3.iterrows(), 1):
    print(f"  {idx}. {row['stock_name']}: ¥{row['profit_per_100_shares_phase2']:+,.0f} ({row['phase2_return']*100:+.2f}%)")
print()
print("ワースト3銘柄:")
worst3 = df_phase2.nsmallest(3, 'profit_per_100_shares_phase2')[['stock_name', 'profit_per_100_shares_phase2', 'phase2_return']]
for idx, (_, row) in enumerate(worst3.iterrows(), 1):
    print(f"  {idx}. {row['stock_name']}: ¥{row['profit_per_100_shares_phase2']:+,.0f} ({row['phase2_return']*100:+.2f}%)")
print()
print("=" * 60)
print("結論:")
if df_phase2['profit_per_100_shares_phase2'].mean() > 0:
    print(f"✅ v1.3は平均して利益が出ています（¥{df_phase2['profit_per_100_shares_phase2'].mean():+,.0f}/100株）")
else:
    print(f"❌ v1.3は平均して損失が出ています（¥{df_phase2['profit_per_100_shares_phase2'].mean():+,.0f}/100株）")
    
print()
print("⚠️ 注意事項:")
print("  - サンプル数が少ない（2日分、25銘柄）")
print("  - より長期間のデータで検証が必要")
print("  - v1.2との比較が必要（localhost:3000/dev/analyzeで確認可能）")
print("=" * 60)

v1.3 Zero Label Strategy: バックテスト結果サマリー

検証期間: 2025-11-05, 2025-11-06（2日分）
検証銘柄数: 25銘柄

Phase2（寄り付き買い → 大引け売り）:
  平均リターン: +0.85%
  平均利益: ¥-134/100株
  勝率: 36.0% (9勝16敗)

全25銘柄に100株ずつ投資した場合:
  合計損益: ¥-3,350

ベスト3銘柄:
  1. 第一実業: ¥+23,100 (+8.72%)
  2. IMV: ¥+6,800 (+3.43%)
  3. マイネット: ¥+5,400 (+24.22%)

ワースト3銘柄:
  1. 大林組: ¥-16,050 (-5.42%)
  2. リクルート: ¥-8,400 (-1.15%)
  3. フルヤ金属: ¥-6,800 (-2.41%)

結論:
❌ v1.3は平均して損失が出ています（¥-134/100株）

⚠️ 注意事項:
  - サンプル数が少ない（2日分、25銘柄）
  - より長期間のデータで検証が必要
  - v1.2との比較が必要（localhost:3000/dev/analyzeで確認可能）


## 11. 結論：v1.3で金が増えるか？

In [66]:
# ゾーン別の平均利益（棒グラフ）
zone_avg = df_phase2.groupby('zone')['profit_per_100_shares_phase2'].mean().reset_index()
zone_avg.columns = ['zone', 'avg_profit']

fig = px.bar(
    zone_avg,
    x='zone',
    y='avg_profit',
    title="ゾーン別 平均利益（Phase2: 100株あたり）",
    labels={'zone': 'ゾーン', 'avg_profit': '平均利益（¥/100株）'},
    color='zone',
    color_discrete_map={
        'Optimal': 'green',
        'Safe': 'lightblue',
        'Danger': 'red'
    },
    text='avg_profit'
)

fig.update_traces(texttemplate='¥%{text:+,.0f}', textposition='outside')
fig.add_hline(y=0, line_dash="dash", line_color="black")
fig.update_layout(height=500, showlegend=False)
fig.update_yaxes(tickprefix="¥", tickformat=",")
fig.show()

In [67]:
# ゾーン別のパフォーマンス
df_phase2['zone'] = df_phase2.apply(
    lambda row: 'Optimal' if (20 <= row['twitter_mentions'] <= 50 and row['previous_day_change_pct'] < 16)
    else ('Danger' if (row['twitter_mentions'] >= 80 or row['previous_day_change_pct'] >= 16)
    else 'Safe'),
    axis=1
)

zone_performance = df_phase2.groupby('zone').agg({
    'profit_per_100_shares_phase2': ['count', 'mean', 'sum'],
    'phase2_win': 'sum'
}).round(0)

zone_performance.columns = ['銘柄数', '平均利益（¥/100株）', '合計利益（¥/100株）', '勝ち銘柄数']
zone_performance['勝率(%)'] = (zone_performance['勝ち銘柄数'] / zone_performance['銘柄数'] * 100).round(1)
zone_performance = zone_performance.sort_values('平均利益（¥/100株）', ascending=False)

print("ゾーン別パフォーマンス:")
print(zone_performance)
print()

# 各銘柄に100株ずつ投資した場合の合計
total_investment = len(df_phase2)
total_profit = df_phase2['profit_per_100_shares_phase2'].sum()
print(f"\n全{total_investment}銘柄に100株ずつ投資した場合:")
print(f"  合計利益: ¥{total_profit:+,.0f}")
print(f"  1銘柄あたり平均: ¥{total_profit/total_investment:+,.0f}")

ゾーン別パフォーマンス:
         銘柄数  平均利益（¥/100株）  合計利益（¥/100株）  勝ち銘柄数  勝率(%)
zone                                                  
Danger     3        1800.0        5400.0      1   33.3
Safe       9         150.0        1350.0      4   44.4
Optimal   13        -777.0      -10100.0      4   30.8


全25銘柄に100株ずつ投資した場合:
  合計利益: ¥-3,350
  1銘柄あたり平均: ¥-134
